<a href="https://colab.research.google.com/github/Shai670/Residential_consumption_modeling/blob/main/Battery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from IPython.display import clear_output 

import matplotlib.pyplot as plt